In [5]:
import pandas as pd
import pydeck
import psycopg2 as pg
from psycopg2.extras import DictCursor


import configparser

config = configparser.ConfigParser()
config.read("../config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

In [8]:
sql = """
select h3id, population FROM h3demographics WHERE
groupname = 'Native Hawaiian and Other Pacific Islander' and population > 0
"""

with pg.connect(**db_params) as conn:
    with conn.cursor(cursor_factory=DictCursor) as cur:
        cur.execute(sql)   
        data = cur.fetchall()

In [11]:
df = pd.DataFrame(data, columns=['h3id', 'pop'])

In [ ]:
sql = """
select h3id, population FROM h3demographics WHERE
groupname = 'Native Hawaiian and Other Pacific Islander' and population > 0
"""

In [14]:
view_state = pydeck.ViewState(
    **{"latitude": 33.7, "longitude": -84.3, "zoom": 10, "maxZoom": 16, "pitch": 0, "bearing": 0}
)

h3_layer = pydeck.Layer(    
    "H3HexagonLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="h3id",    
    get_line_color=[0, 0, 0],
    get_fill_color=[100, 100, 100],
    line_width_min_pixels=2,
)

r = pydeck.Deck(
    [h3_layer],
    initial_view_state=view_state,    
    map_style=pydeck.map_styles.LIGHT,
)
r.to_html()